In [1]:
import spacy
from spacy import displacy

import pandas as pd
from spacy.matcher import Matcher

nlp  = spacy.load('en_core_web_sm')

In [13]:
output_path = "./data/output/"
ner_pickles_op = output_path + "ner/"
cr_pickles_op = output_path + "cr/"

''' read restaurant knowledgebase data '''
knowledge_base_entities = r"data/kb/entities/"
kb_restaurant_file = knowledge_base_entities + "restaurant.csv"
kb_menu_file = knowledge_base_entities + "menu.csv"
kb_menu_file1 = knowledge_base_entities + "menu1.csv"
kb_general_file = knowledge_base_entities + "general.csv"
kb_restaurant_aspects_file = knowledge_base_entities + "restaurant_aspects.csv"
kb_menu_aspects_file = knowledge_base_entities + "menu_attrs.csv"

entity_headers = ['Name', 'Label']


In [11]:
# Changing the dataset
# i.e.
###
#   Silver Spoon,b-nbf4o8gUP2y9JKFE2wBO2A,HAS_MENU,Butter Chicken,Butter_Chicken
#   Great,Great,MENU_ATTR_FOR,Silver Spoon,b-nbf4o8gUP2y9JKFE2wBO2A
#   Butter Chicken,Butter_Chicken,IS,Great,Great
#
#   Silver Spoon,b-nbf4o8gUP2y9JKFE2wBO2A,HAS_ASPECT,Staff,Staff
#   Nice,Nice,ASPECT_ATTR_FOR,Silver Spoon,b-nbf4o8gUP2y9JKFE2wBO2A
#   Staff,Staff,IS,Nice,Nice
###

import pandas as pd
df = pd.read_csv("data/output/kg/input_data.txt-out2.csv", sep=",")

for index, row in df.iterrows():
    if row["relation"] == "HAS_MENU":
        # print(row["relation"])
        # print(df.loc[index]["relation"])
        # print(df.loc[index+1]["relation"])
        # print(df.loc[index+2]["relation"])
        # print("######")

        if df.loc[index+1]["relation"] == "MENU_ATTR_FOR" and \
            df.loc[index+2]["relation"] == "IS":
            df.loc[index+1]["subject"] = df.loc[index]["object"]+"_"+df.loc[index+1]["subject"]
            df.loc[index+2]["object"] = df.loc[index+1]["subject"]

    if row["relation"] == "HAS_ASPECT":
        if df.loc[index+1]["relation"] == "ASPECT_ATTR_FOR" and \
            df.loc[index+2]["relation"] == "IS":
            df.loc[index+1]["subject"] = df.loc[index]["object"]+"_"+df.loc[index+1]["subject"]
            df.loc[index+2]["object"] = df.loc[index+1]["subject"]

for index, row in df.iterrows():
    if row["relation"] == "MENU_ATTR_FOR":
        if df.loc[index+1]["relation"] == "IS":
            if df.loc[index]["subject"] != '_'.join([s for s in df.loc[index+1]["sub_ent"].split()]) +"_"+'_'.join([s for s in df.loc[index]["sub_ent"].split()]):
                df.loc[index]["subject"] = df.loc[index+1]["subject"]+"_"+df.loc[index]["subject"]
                df.loc[index+1]["object"] = df.loc[index]["subject"]

    if row["relation"] == "ASPECT_ATTR_FOR":
        if df.loc[index+1]["relation"] == "IS":
            if df.loc[index]["subject"] != '_'.join([s for s in df.loc[index+1]["sub_ent"].split()]) +"_"+'_'.join([s for s in df.loc[index]["sub_ent"].split()]):
                df.loc[index]["subject"] = df.loc[index+1]["subject"]+"_"+df.loc[index]["subject"]
                df.loc[index+1]["object"] = df.loc[index]["subject"]

df.to_csv("data/output/kg/input_data.txt-out4.csv", index=False)


In [16]:
''' make dataframes for kb '''
kb_restaurant_df = pd.read_csv(kb_restaurant_file, header=0, names=entity_headers)
kb_menu_df = pd.read_csv(kb_menu_file, header=0, names=entity_headers)
# kb_general_df = pd.read_csv(kb_general_file, header=0, names=['Name', 'Label'])
kb_restaurant_aspects_df = pd.read_csv(kb_restaurant_aspects_file, header=0, names=entity_headers)
kb_menu_aspects_df = pd.read_csv(kb_menu_aspects_file, header=0, names=entity_headers)



In [2]:
kb_entities = {
    "Category": ["Chinese", "Indian", "Korean", "Italian", "Mexican"],
    "Attr": ["Good", "Sweet", "Spicy", "Friendly"],
    "Aspect": ["Service", "Take out", "Staff", "Parking"],
    "City": ["Toronto", "Seoul", "New York"],
    "Menu": ["Noodles", "Chicken Biryani", "Kimchi"]
}
# "", "find", "search", "recommend", "suggest" __ "restaurant|restaurants"
# "", "what", "which" __ "menu|menus"
kb_rel = {
    "Category": {"in": ["HAS_CATEGORY"], "out": []},
    "Attr": {"in": ["IS"], "out": ["MENU_ATTR_FOR"]},
    "Aspect": {"in": ["HAS_ASPECT"], "out": ["ASPECT_ATTR_FOR", "IS"]},
    "City": {"in": ["LOCATED_IN"], "out": ["LOCATED_IN"]},
    "Menu": {"in": ["HAS_MENU", "ORDER"], "out": ["IS"]}
}

In [12]:
text = "Recommend best Chinese restaurant in Toronto or seoul which serves sweet Noodles and spicy Chicken Biryani"


doc = nlp(text)
print([(ent.text, ent.label_) for ent in doc.ents])

# verbs = [tok for tok in doc if tok.pos_ == "VERB"]
toks = [tok.pos_ for tok in doc]
print(toks)

triplets = []


[('Chinese', 'NORP'), ('Toronto', 'GPE'), ('seoul', 'GPE'), ('Chicken Biryani', 'PERSON')]
['VERB', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'PROPN', 'CCONJ', 'PROPN', 'DET', 'VERB', 'ADJ', 'NOUN', 'CCONJ', 'NOUN', 'PROPN', 'PROPN']


In [8]:
text = "Recommend best Chinese restaurant in Toronto or seoul which serves sweet Noodles and spicy Chicken Biryani"
doc = nlp(text)

verbs = [tok for tok in doc if tok.pos_ == "CCONJ"]

# getting sub, verb, obj triples
# Angela, visit, silver spoon restaurant
for v in verbs:
    # print(v.lefts)
    # print(v.rights)
    # for tok in v.lefts:
        # print(tok)
    # print([token.text for token in doc[2].lefts])  # ['bright', 'red']
    # print([token.text for token in doc[2].rights])  # ['on']

    print(v.i)
    left_tok = [tok.text for tok in doc[v.i].lefts]
    right_tok = [tok.text for tok in v.rights]

    print(left_tok)
    print(right_tok)

6
[]
[]
12
[]
[]


In [6]:
text = "Recommend best Chinese restaurant in Toronto or seoul which serves sweet Noodles and spicy Chicken Biryani"

# Plot the dependency graph
doc = nlp(text)
displacy.render(doc, style='dep',jupyter=True)

In [8]:
text = "Recommend best Chinese restaurant in Toronto or New York which serves sweet Noodles and spicy Chicken Biryani"

# Plot the dependency graph
doc = nlp(text)
displacy.render(doc, style='ent',jupyter=True)

In [34]:
from spacy.tokens import Span

text = "Recommend best Chinese restaurant in Toronto or New York and seoul which serves sweet Noodles and spicy Chicken Biryani"
doc = nlp(text)
# print(doc.ents)
#
# for tok in doc:
#   print(tok.text, tok.dep_, tok.pos_)

location = []
for ent in doc.ents:
    if ent.label_ == "GPE" and ent.start != 0:
            prev_token = doc[ent.start - 1]
            if prev_token.text in ("and", "or"):
                location.append({"op":prev_token.text.upper(), "ent":ent.text.title()})
            else:
                location.append({"op":None, "ent":ent.text.title()})
print(location)


def kb_lookup(dictionary, string):
    return {key:val for key,val in dictionary.items() if any(string in s for s in val)}

menus = kb_lookup(kb_entities, text)
print(menus)

[{'op': None, 'ent': 'Toronto'}, {'op': 'OR', 'ent': 'New York'}, {'op': 'AND', 'ent': 'Seoul'}]
{}


In [41]:
from spacy.vocab import Vocab
vocab = Vocab(strings=["Butter Chicken", "Chicken Biryani"])

text = "Recommend best Chinese restaurant in Toronto or New York and seoul which serves sweet Noodles and spicy Chicken Biryani"
doc = nlp(text)

def kb_lookup(dictionary, string):
    concepts = []
    for key, val in dictionary.items():
        if key != "Attr":
            for concept in val:
                if concept.lower() in string.lower():
                    # if key == "Menu":
                    #     print(string.index(concept))
                    concepts.append({key: concept})
    return concepts

concepts = kb_lookup(kb_entities, text)
print(concepts)

patterns = []
for concept in concepts:
    for k,  v in concept.items():
        if k == "Menu":
            patterns.append([{"TEXT": v}])

print(patterns)
assert "Noodles" in nlp.vocab
matcher = Matcher(vocab)

matcher.add("menu_patter", None, patterns[0])
# Use the matcher on the doc
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

[{'Category': 'Chinese'}, {'City': 'Toronto'}, {'City': 'Seoul'}, {'City': 'New York'}, {'Menu': 'Noodles'}, {'Menu': 'Chicken Biryani'}]
[[{'TEXT': 'Noodles'}], [{'TEXT': 'Chicken Biryani'}]]
Matches: ['Noodles']
